# Lab 4: Variational Inference with a Fully-Factorized Gaussian Variational Distribution

In [43]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions

# Naval Propulsion Model

<span>1.</span>Train a Bayesian neural network using variational inference, a fully-factorized Gaussian variational distribution,
and a Gaussian prediction with heteroscedastic uncertainty for the Naval Propulsion dataset.

In [44]:
naval_data = pd.read_csv('../data/naval.csv')

In [45]:
naval_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11934 entries, 0 to 11933
Data columns (total 18 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   1 - Lever position (lp) [ ]                         11934 non-null  float64
 1   2 - Ship speed (v) [knots]                          11934 non-null  float64
 2   3 - Gas Turbine shaft torque (GTT) [kN m]           11934 non-null  float64
 3   4 - Gas Turbine rate of revolutions (GTn) [rpm]     11934 non-null  float64
 4   5 - Gas Generator rate of revolutions (GGn) [rpm]   11934 non-null  float64
 5   6 - Starboard Propeller Torque (Ts) [kN]            11934 non-null  float64
 6   7 - Port Propeller Torque (Tp) [kN]                 11934 non-null  float64
 7   8 - HP Turbine exit temperature (T48) [C]           11934 non-null  float64
 8   9 - GT Compressor inlet air temperature (T1) [C]    11934 non-null  float64


In [46]:
y = naval_data[naval_data.columns[16]]
X = naval_data.iloc[:,0:16]
y.info()
print(y)
X.info()

<class 'pandas.core.series.Series'>
RangeIndex: 11934 entries, 0 to 11933
Series name: 17 - GT Compressor decay state coefficient.
Non-Null Count  Dtype  
--------------  -----  
11934 non-null  float64
dtypes: float64(1)
memory usage: 93.4 KB
0        0.95
1        0.95
2        0.95
3        0.95
4        0.95
         ... 
11929    1.00
11930    1.00
11931    1.00
11932    1.00
11933    1.00
Name: 17 - GT Compressor decay state coefficient., Length: 11934, dtype: float64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11934 entries, 0 to 11933
Data columns (total 16 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   1 - Lever position (lp) [ ]                         11934 non-null  float64
 1   2 - Ship speed (v) [knots]                          11934 non-null  float64
 2   3 - Gas Turbine shaft torque (GTT) [kN m]           11934 non-null  float64
 3   4 

In [47]:
from sklearn.preprocessing import StandardScaler

scaler_X = StandardScaler()
scaler_y = StandardScaler()

X = scaler_X.fit_transform(X)
y = y.to_numpy()
y = y.reshape(-1,1)
y = scaler_y.fit_transform(y)

print(X)
print(y)

[[-1.53324812 -1.54919334 -1.2166697  ... -0.9486833  -1.02557391
  -1.14440517]
 [-1.17161304 -1.161895   -0.91565197 ... -0.9486833  -0.88780617
  -0.7403042 ]
 [-0.77191112 -0.77459667 -0.85156724 ... -0.9486833  -0.79492904
  -0.79549848]
 ...
 [ 0.75456956  0.77459667  0.53031625 ...  0.9486833   0.32733629
   0.33795544]
 [ 1.15807816  1.161895    1.07187738 ...  0.9486833   0.94651716
   0.96086229]
 [ 1.57300683  1.54919334  2.05571343 ...  1.8973666   2.03008369
   2.04503561]]
[[-1.69841555]
 [-1.69841555]
 [-1.69841555]
 ...
 [ 1.69841555]
 [ 1.69841555]
 [ 1.69841555]]


In [48]:
n_examples = y.shape[0]
expectNLL = lambda y, rv_y: -rv_y.log_prob(y)/n_examples

In [49]:
prior = tfp.layers.default_mean_field_normal_fn(
    is_singular=False,
    loc_initializer=tf.zeros_initializer(),
    untransformed_scale_initializer=tf.constant_initializer(0.01),
    loc_regularizer=None,
    untransformed_scale_regularizer=None,
    loc_constraint=None,
    untransformed_scale_constraint=None
)

In [50]:
def get_kernel_divergence_fn(train_size, kl_weight=1.0):
  def kernel_divergence_fn(q, p, _):
    kernel_divergence = tfp.distributions.kl_divergence(q, p) / tf.cast(train_size, tf.float32)
    return kl_weight * kernel_divergence
  return kernel_divergence_fn

In [51]:
kl_div = get_kernel_divergence_fn(y.shape[0])

In [52]:
input = tf.keras.layers.Input(16)
h1 = tfp.layers.DenseLocalReparameterization(100,activation=tf.keras.activations.relu,kernel_prior_fn=prior,bias_prior_fn=prior,kernel_divergence_fn=kl_div,bias_divergence_fn=kl_div)(input)
m = tfp.layers.DenseLocalReparameterization(1,kernel_prior_fn=prior,bias_prior_fn=prior,kernel_divergence_fn=kl_div,bias_divergence_fn=kl_div)(h1)
s = tfp.layers.DenseLocalReparameterization(1,activation=tf.keras.activations.softplus,kernel_prior_fn=prior,bias_prior_fn=prior,kernel_divergence_fn=kl_div,bias_divergence_fn=kl_div)(h1)
p = tfp.layers.DistributionLambda(lambda t: tfd.Normal(loc=t[0], scale=t[1]))([m,s])
model = tf.keras.Model(inputs=input, outputs=p, name="NavalModel")

model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.00000001), loss=expectNLL)
print(model.summary())

/home/alexander.huang/.conda/envs/py38/lib/python3.8/site-packages/tensorflow_probability/python/layers/util.py:95: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use the `layer.add_weight()` method instead.
  loc = add_variable_fn(
/home/alexander.huang/.conda/envs/py38/lib/python3.8/site-packages/tensorflow_probability/python/layers/util.py:105: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use the `layer.add_weight()` method instead.
  untransformed_scale = add_variable_fn(


Model: "NavalModel"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 16)]         0           []                               
                                                                                                  
 dense_local_reparameterization  (None, 100)         6700        ['input_6[0][0]']                
 _15 (DenseLocalReparameterizat                                                                   
 ion)                                                                                             
                                                                                                  
 dense_local_reparameterization  (None, 1)           403         ['dense_local_reparameterization_
 _16 (DenseLocalReparameterizat                                  15[0][0]']              

In [53]:
n_models = 10
predictions = []
means = []

for _ in range(n_models):
    model.fit(X, y, epochs=10, verbose=False)
    yhat = model.predict(X)
    yhat = scaler_y.inverse_transform(yhat)
    predictions.append(yhat)
    y_dist = model(X)
    mean = y_dist.mean().numpy()
    means.append(mean)

373/373 [==============================] - 1s 2ms/step


<span>2.</span>Find the epistemic uncertainty for all Naval Propulsion examples.

In [54]:
result = 0
result = np.var(means)
print('Epistemic Uncertainty', result)

Epistemic Uncertainty 0.043565214


In [55]:
from sklearn.metrics import mean_squared_error
# print RMSE from predictions of one of the models
print('RMSE:', np.sqrt(mean_squared_error(y, predictions[0])))

RMSE: 1.39465003810513


# Ionosphere Model

<span>3.</span>Train a Bayesian neural network using variational inference, a fully-factorized Gaussian variational distribution, and a Bernoulli prediction for the Ionosphere dataset.

In [56]:
ion_data = pd.read_csv('../data/ionosphere.csv', header=None)
print(ion_data.info)

<bound method DataFrame.info of      0   1        2        3        4        5        6        7        8   \
0     1   0  0.99539 -0.05889  0.85243  0.02306  0.83398 -0.37708  1.00000   
1     1   0  1.00000 -0.18829  0.93035 -0.36156 -0.10868 -0.93597  1.00000   
2     1   0  1.00000 -0.03365  1.00000  0.00485  1.00000 -0.12062  0.88965   
3     1   0  1.00000 -0.45161  1.00000  1.00000  0.71216 -1.00000  0.00000   
4     1   0  1.00000 -0.02401  0.94140  0.06531  0.92106 -0.23255  0.77152   
..   ..  ..      ...      ...      ...      ...      ...      ...      ...   
346   1   0  0.83508  0.08298  0.73739 -0.14706  0.84349 -0.05567  0.90441   
347   1   0  0.95113  0.00419  0.95183 -0.02723  0.93438 -0.01920  0.94590   
348   1   0  0.94701 -0.00034  0.93207 -0.03227  0.95177 -0.03431  0.95584   
349   1   0  0.90608 -0.01657  0.98122 -0.01989  0.95691 -0.03646  0.85746   
350   1   0  0.84710  0.13533  0.73638 -0.06151  0.87873  0.08260  0.88928   

          9   ...       25     

In [57]:
"""
# One hot, label is last two columns
def one_hot(cat, hot):
    if cat == hot:
        return int(1)
    else:
        return int(0)
    
ion_data[35] = ion_data[34].apply(one_hot, hot='g')
ion_data[36] = ion_data[34].apply(one_hot, hot='b')
"""

def ordinal_encoder(category):
    dict = {'g': 0, 'b': 1}
    return dict[category]

print('g class:', ordinal_encoder('g'))
print('b class:', ordinal_encoder('b'))
ion_data[34] = ion_data[34].apply(ordinal_encoder)
print(ion_data[34].to_numpy())

g class: 0
b class: 1
[0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0
 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1
 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 0 1 0 1 0 1 0 1 0 1 0 1 0 1
 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0
 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1
 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0
 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [58]:
# split x and y
x = ion_data.iloc[:,0:33].to_numpy()
y = ion_data.iloc[:,34].to_numpy().reshape(-1,1)

# Scaling
scaler = StandardScaler()
x = scaler.fit_transform(x)
print(x.shape)
print(y.shape)

(351, 33)
(351, 1)


In [63]:
input2 = tf.keras.layers.Input(33)
h21 = tfp.layers.DenseLocalReparameterization(100,activation=tf.keras.activations.relu,kernel_prior_fn=prior,bias_prior_fn=prior,kernel_divergence_fn=kl_div,bias_divergence_fn=kl_div)(input2)
p_val = tfp.layers.DenseLocalReparameterization(1,kernel_prior_fn=prior,bias_prior_fn=prior,kernel_divergence_fn=kl_div,bias_divergence_fn=kl_div)(h21)
out2 = tfp.layers.DistributionLambda(lambda t: tfp.distributions.Bernoulli(probs=t,  dtype=tf.float64), name="output")(p_val)

model2 = tf.keras.Model(inputs=input2, outputs=out2, name="IonosphereNNMLEwDistr")
 
model2.compile(optimizer=tf.optimizers.Adam(learning_rate=0.0000001), loss=expectNLL)
print(model2.summary())

/home/alexander.huang/.conda/envs/py38/lib/python3.8/site-packages/tensorflow_probability/python/layers/util.py:95: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use the `layer.add_weight()` method instead.
  loc = add_variable_fn(
/home/alexander.huang/.conda/envs/py38/lib/python3.8/site-packages/tensorflow_probability/python/layers/util.py:105: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use the `layer.add_weight()` method instead.
  untransformed_scale = add_variable_fn(


Model: "IonosphereNNMLEwDistr"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 33)]              0         
                                                                 
 dense_local_reparameterizat  (None, 100)              13500     
 ion_20 (DenseLocalReparamet                                     
 erization)                                                      
                                                                 
 dense_local_reparameterizat  (None, 1)                403       
 ion_21 (DenseLocalReparamet                                     
 erization)                                                      
                                                                 
 output (DistributionLambda)  ((None, 1),              0         
                              (None, 1))                         
                                             

In [64]:
n_models = 10
predictions = []
means = []

for _ in range(n_models):
    model2.fit(x, y, epochs=50, verbose=False)
    yhat2 = model2.predict(x)
    yhat2 = scaler_y.inverse_transform(yhat)
    predictions.append(yhat)
    y_dist2 = model2(x)
    mean = y_dist2.mean().numpy()
    means.append(mean)

11/11 [==============================] - 0s 2ms/step


In [65]:
from sklearn.metrics import classification_report

yhat3 = model2.predict(x)

def labelmaker(prob):
    if prob > 0.4:
        return 1
    else:
        return 0

yhat3 = np.apply_along_axis(labelmaker, 1, yhat3)
print(classification_report(y.flatten(), yhat3.flatten()))

11/11 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

           0       0.70      0.97      0.81       225
           1       0.82      0.25      0.38       126

    accuracy                           0.71       351
   macro avg       0.76      0.61      0.59       351
weighted avg       0.74      0.71      0.66       351



<span>4.</span> Find the epistemic uncertainty for all Ionosphere examples.

In [69]:
result = 0
result = np.var(means)
print('Epistemic Uncertainty', result)

Epistemic Uncertainty 0.1577162
